In [114]:
from itertools import product
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [115]:
dim_cell = 3
range_cell = range(1, dim_cell + 1) 
dim_board = dim_cell * dim_cell
range_board = range(1, dim_board + 1)

In [116]:
board = {(i, j): model.NewIntVar(1, dim_board, f'field_{i}_{j}')
         for i, j in product(range_board, repeat=2)}

In [117]:
cell_iter = [product([offset_i * dim_cell + i for i in range_cell], 
                 [offset_j * dim_cell + j for j in range_cell])
            for offset_i in range(dim_cell)
            for offset_j in range(dim_cell)] 
cells = [[board[(i, j)] for i, j in cell] for cell in cell_iter] 

In [118]:
for ind in range_board:
    model.AddAllDifferent(board[(ind, j)] for j in range_board)
    model.AddAllDifferent(board[(i, ind)] for i in range_board)

In [119]:
init_sol = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [120]:
for cell in cells:
    model.AddAllDifferent(cell)        

In [121]:
for i, j in product(range_board, repeat=2):
    if init_sol[i-1][j-1] != 0:
        model.Add(board[(i,j)] == init_sol[i-1][j-1])

In [122]:
solver = cp_model.CpSolver()

In [125]:
%%time
res = solver.Solve(model)

CPU times: user 3.46 ms, sys: 776 µs, total: 4.23 ms
Wall time: 3.7 ms


In [126]:
if res == cp_model.FEASIBLE:
    for i in range_board:
        print([solver.Value(board[(i, j)]) for j in range_board])
else:
    print("Could not solve puzzle!")

[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
